In [ ]:
import numpy as np
import librosa
import cv2
import moviepy.editor as mpy

def create_particles(num_particles, width, height):
    particles = np.random.rand(num_particles, 4)
    particles[:, 0] *= width
    particles[:, 1] *= height
    particles[:, 2] = np.random.randint(2, 5, num_particles)
    particles[:, 3] = np.random.randint(1, 5, num_particles)
    return particles

def update_particles(particles, width, height, energy):
    particles[:, 1] -= particles[:, 3] * energy
    reset = particles[:, 1] < 0
    particles[reset, 1] = height
    particles[reset, 0] = np.random.randint(0, width, np.sum(reset))
    return particles

def draw_particles(frame, particles, color):
    for x, y, size, _ in particles:
        cv2.circle(frame, (int(x), int(y)), int(size), color, -1)

def apply_blur_effect(frame, energy):
    blur_amount = max(1, int(energy * 2))
    return cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

def apply_bounce_effect(frame, current_onset, max_onset, original_height):
    bounce_amount = int(30 * current_onset / max_onset)
    if bounce_amount > 0:
        frame = frame[bounce_amount:-bounce_amount, :]
    return cv2.resize(frame, (frame.shape[1], original_height))

def apply_glitch_effect(frame, strength=10):
    height, width, _ = frame.shape
    glitch_frame = frame.copy()
    num_slices = np.random.randint(1, strength)
    for _ in range(num_slices):
        slice_height = np.random.randint(1, height // strength)
        start_y = np.random.randint(0, height - slice_height)
        start_x = np.random.randint(-strength, strength)
        end_x = width + start_x
        if start_x > 0:
            end_x = min(end_x, width)
            glitch_frame[start_y:start_y + slice_height, start_x:end_x] = frame[start_y:start_y + slice_height, :end_x - start_x]
        else:
            start_x = max(start_x, -width)
            glitch_frame[start_y:start_y + slice_height, :end_x] = frame[start_y:start_y + slice_height, -start_x:]
    return glitch_frame

def apply_trailing_effect(frame, previous_frames):
    alpha = 0.7
    result = frame.copy()
    for prev_frame in previous_frames:
        result = cv2.addWeighted(result, alpha, prev_frame, 1 - alpha, 0)
    return result

def apply_color_shift(frame, shift_amount):
    b, g, r = cv2.split(frame)
    b = np.roll(b, shift_amount, axis=1)
    r = np.roll(r, -shift_amount, axis=1)
    return cv2.merge([b, g, r])

def draw_geometric_shapes(frame, energy, height, width):
    num_shapes = int(energy * 10)
    for _ in range(num_shapes):
        shape_type = np.random.choice(['circle', 'rectangle', 'line'])
        color = (255, 255, 255)
        if shape_type == 'circle':
            center = (np.random.randint(0, width), np.random.randint(0, height))
            radius = np.random.randint(5, 30)
            cv2.circle(frame, center, radius, color, 2)
        elif shape_type == 'rectangle':
            top_left = (np.random.randint(0, width), np.random.randint(0, height))
            bottom_right = (top_left[0] + np.random.randint(20, 50), top_left[1] + np.random.randint(20, 50))
            cv2.rectangle(frame, top_left, bottom_right, color, 2)
        elif shape_type == 'line':
            pt1 = (np.random.randint(0, width), np.random.randint(0, height))
            pt2 = (np.random.randint(0, width), np.random.randint(0, height))
            cv2.line(frame, pt1, pt2, color, 2)

def create_music_visualizer(image_path, audio_path, output_path):
    y, sr = librosa.load(audio_path)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    duration = librosa.get_duration(y=y, sr=sr)
    fps = 30
    n_frames = int(duration * fps)
    
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=32, fmax=8000, hop_length=sr//fps)
    S_dB = librosa.power_to_db(S, ref=np.max)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=sr//fps)
    
    height, width, _ = img.shape
    particles = create_particles(100, width, height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('temp_output.mp4', fourcc, fps, (width, height))
    
    prev_bars = np.zeros(32)
    zoom_factor = 1.0
    max_onset = np.max(onset_env)
    previous_frames = []

    for frame_num in range(n_frames):
        audio_idx = int(frame_num * len(y) / n_frames)
        chunk = y[audio_idx:audio_idx + sr//fps]
        
        spec_frame = S_dB[:, min(frame_num, S_dB.shape[1]-1)]
        current_onset = onset_env[min(frame_num, len(onset_env)-1)]
        energy = np.mean(np.abs(chunk)) * 10

        frame = img.copy()
        particles = update_particles(particles, width, height, energy)
        draw_particles(frame, particles, (255, 255, 255))

        bar_width = width // 32
        max_bar_height = height // 4

        target_heights = np.interp(spec_frame, [S_dB.min(), S_dB.max()], [0, max_bar_height])
        prev_bars = prev_bars * 0.7 + target_heights * 0.3
        bar_heights = prev_bars.astype(int)

        for j, bar_height in enumerate(bar_heights):
            cv2.rectangle(frame,
                          (j * bar_width, height // 2 - bar_height),
                          ((j + 1) * bar_width, height // 2),
                          (255, 255, 255, 150),
                          -1)
            cv2.rectangle(frame,
                          (j * bar_width, height // 2),
                          ((j + 1) * bar_width, height // 2 + bar_height),
                          (255, 255, 255, 150),
                          -1)

        frame = apply_blur_effect(frame, energy)
        target_zoom = 1 + 0.05 * current_onset / max_onset
        zoom_factor = zoom_factor * 0.7 + target_zoom * 0.3
        scaled_frame = cv2.resize(frame, None, fx=zoom_factor, fy=zoom_factor)
        start_y = (scaled_frame.shape[0] - height) // 2
        start_x = (scaled_frame.shape[1] - width) // 2
        frame = scaled_frame[start_y:start_y + height, start_x:start_x + width]
        frame = apply_bounce_effect(frame, current_onset, max_onset, height)
        
        frame = apply_color_shift(frame, int(energy * 10))
        
        if np.random.rand() < 0.1:
            frame = apply_glitch_effect(frame)
        
        if len(previous_frames) > 5:
            previous_frames.pop(0)
        previous_frames.append(frame.copy())
        frame = apply_trailing_effect(frame, previous_frames)

        draw_geometric_shapes(frame, energy, height, width)
        
        video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    video.release()
    
    video = mpy.VideoFileClip('temp_output.mp4')
    audio = mpy.AudioFileClip(audio_path).set_duration(video.duration)
    final_video = video.set_audio(audio)
    final_video.write_videofile(output_path, codec="libx264", audio_codec="aac")

# Paths to the image and audio files
image_path = 'image.png'
audio_path = 'audio.mp3'  
output_path = 'output_videoDemo.mp4'
create_music_visualizer(image_path, audio_path, output_path)

